**Computes large population movements**

**Input**: csv using the following columns:
* `transportation_mode`: used mode of transport for the trip,
* `starting_longitude` and `starting_latitude`: Starting point of trip
* `ending_longitude` and `ending_latitude`: Ending point of trip
* `user_id`: Id of the traveling user, used to make sure results include more than 3 users per geographic division
* `journey_id`: id of journeys grathering multiple trips
* `start_time`: Begining date of trip, can be used to compute trip duration
* `end_time`: End date of trip, can be used to compute trip duration

**Input**: geoJSON file with perimeter of Ile-de-France region

**Output**: geoJSON file "exode.geojson" containing h3 cell shapes with the following metadata:
* `geometry`: h3 shape of the destinations
* `MostCommonTransport`: prefered `transportation_mode_tr` for most trips
* `color`: A color representation of `MostCommonTransport`
* `Count`: Total number of trips arriving in cell from Ile-de-France

**Output**: geoJSON file "exode_lines.geojson" containing line shapes with the following metadata:
* `geometry`: LineString starting from Ile-de-France and ending in the centroid of h3 celles
* `MostCommonTransport`: prefered `transportation_mode_tr` for most trips
* `color`: A color representation of `MostCommonTransport`
* `Count`: Total number of trips arriving in cell from Ile-de-France

**Output**: geoJSON file "inxode.geojson" containing h3 cell shapes with the following metadata:
* `geometry`: h3 shape of the origins
* `MostCommonTransport`: prefered `transportation_mode_tr` for most trips
* `color`: A color representation of `MostCommonTransport`
* `Count`: Total number of trips leaving from cell to Ile-de-France

**Output**: geoJSON file "inxode_lines.geojson" containing line shapes with the following metadata:
* `geometry`: LineString starting from Ile-de-France and ending in the centroid of h3 celles
* `MostCommonTransport`: prefered `transportation_mode_tr` for most trips
* `color`: A color representation of `MostCommonTransport`
* `Count`: Total number of trips leaving from cell to Ile-de-France


The way the MostCommonTransport is computed is as follow:
* We aggregate trips in journeys, 
    * journey start point corresponds to the start point of the first trip
    * journey end point corresponds to the end point of the last trip
    * journey transportation mode corresponds to the most used transportation mode, by distance travelled
* We then aggregate journeys by h3 hexagon cells
    * most common transport is the transportation mode appearing for the most trips in a cell



In [ ]:
# Configuration
INPUT_CSV_FILE = "sources/data_france_2024-07-31.csv" # "sources/data.csv"
ILE_DE_FRANCE_GEOJSON_PERIMETER_FILE = "sources/region-ile-de-france.geojson"
DATE = "2024-07-31"
OUTPUT_FOLDER = "../static/data/exode/" + DATE + "/"
EXODE_OUTPUT_FILE = OUTPUT_FOLDER + "exode.geojson"
EXODE_LINES_OUTPUT_FILE = OUTPUT_FOLDER + "exode_lines.geojson"
INXODE_OUTPUT_FILE = OUTPUT_FOLDER + "inxode.geojson"
INXODE_LINES_OUTPUT_FILE = OUTPUT_FOLDER + "inxode_lines.geojson"

In [ ]:
import pandas as pd
from mappymatch.constructs.geofence import Geofence
from shapely.geometry import Point, LineString
from shapely.vectorized import contains
import h3pandas
import geopandas as gpd
import folium
import json
import numpy as np
import os


In [ ]:
os.makedirs(os.path.dirname(OUTPUT_FOLDER), exist_ok=True)

### Load sources

In [ ]:
geofence_idf = Geofence.from_geojson(ILE_DE_FRANCE_GEOJSON_PERIMETER_FILE)

In [ ]:
df = pd.read_csv(INPUT_CSV_FILE)
df = df.rename(columns={"moover_id": "user_id"})

### Add a human readable transporation mode

In [ ]:
tr = {
-10 : "NOT_DEFINED",
0 : "UNKNOWN",
1 : "PASSENGER_CAR",
2 : "MOTORCYCLE",
3 : "HEAVY_DUTY_VEHICLE",
4 : "BUS",
5 : "COACH",
6 : "RAIL_TRIP",
7 : "BOAT_TRIP",
8 : "BIKE_TRIP",
9 : "PLANE",
10 : "SKI",
11 : "FOOT",
12 : "IDLE",
13 : "OTHER",
101 : "SCOOTER",
102 : "HIGH_SPEED_TRAIN"
}
df['transportation_mode_tr'] = df['transportation_mode'].apply(lambda x: tr[x])

### Consider journeys instead of trips (aggregation)

In [ ]:
# Aggregate the main features
agg_main = df.groupby('journey_id').agg(
    starting_longitude=('starting_longitude', 'first'),
    starting_latitude=('starting_latitude', 'first'),
    start_time=('start_time', 'first'),
    ending_longitude=('ending_longitude', 'last'),
    ending_latitude=('ending_latitude', 'last'),
    end_time=('end_time', 'last'),
    user_id=('user_id', 'first')
).reset_index()

# Calculate the sum of distances for each transportation mode within each journey
agg_distance = df.groupby(['journey_id', 'transportation_mode_tr']).agg(
    total_distance=('distance_km', 'sum')
).reset_index()

# Sort the distance aggregation and find the top two transportation modes for each journey
agg_distance_sorted = agg_distance.sort_values(by=['journey_id', 'total_distance'], ascending=[True, False])

# Get the top two transportation modes for each journey
agg_distance_top2 = agg_distance_sorted.groupby('journey_id').head(2).reset_index(drop=True)

# Split the top two transportation modes into separate columns
agg_distance_top2['rank'] = agg_distance_top2.groupby('journey_id').cumcount() + 1
agg_distance_pivot = agg_distance_top2.pivot(index='journey_id', columns='rank', values=['transportation_mode_tr', 'total_distance']).reset_index()

# Rename columns for clarity
agg_distance_pivot.columns = ['journey_id', 
                              'top_transportation_mode_tr', 'second_top_transportation_mode_tr', 
                              'top_transportation_mode_distance', 'second_top_transportation_mode_distance']

# Merge the results
result = pd.merge(agg_main, agg_distance_pivot, on='journey_id', how='left')

In [ ]:
result["merge_transportation_mode_tr"] = result["top_transportation_mode_tr"] + result["second_top_transportation_mode_tr"].fillna('')
result

In [ ]:
# ignore southern part of the world, data is messed up anyway
df = result[result["ending_latitude"] > 35][result["starting_latitude"] > 35]

# Or consider whole data
#df = result

### Generate Exode data

Those are trips leaving idf

In [ ]:
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.starting_longitude, df.starting_latitude), crs="EPSG:4326")


In [ ]:
geofence_idf_geometry = geofence_idf.geometry

# Keep only trips starting in idf
gdf_from_idf = gdf[gdf.geometry.within(geofence_idf_geometry)]

# Create end_geometry for the end points
gdf_from_idf['end_geometry'] = gpd.points_from_xy(gdf_from_idf.ending_longitude, gdf_from_idf.ending_latitude)

# Remove trips ending in idf
gdf_exit_idf = gdf_from_idf[~gdf_from_idf['end_geometry'].within(geofence_idf_geometry)]

# Drop the temporary 'end_geometry' column if not needed
gdf_exit_idf = gdf_exit_idf.drop(columns=['geometry', 'end_geometry'])
gdf_exit_idf

In [ ]:
# Optional, map colored by count, not an output

# dfh3 = gdf_exit_idf.h3.geo_to_h3(4, lat_col="ending_latitude", lng_col="ending_longitude", set_index=False)
# df_unique_user = dfh3.drop_duplicates(subset=['h3_04', 'user_id'])
# drawgeoframe = df_unique_user[['h3_04']].groupby(['h3_04']).agg(Count=('h3_04', np.size))
# drawgeoframe=drawgeoframe.reset_index().set_index('h3_04')
# drawgeoframe = drawgeoframe[drawgeoframe['Count'] > 3]
# drawgeoframe = drawgeoframe.h3.h3_to_geo()
# drawgeoframe["center_geom"] = drawgeoframe["geometry"]
# drawgeoframe = drawgeoframe.h3.h3_to_geo_boundary()

# fixed_point = Point(2.333333, 48.866667)

# # Function to create a line from the fixed point to each point
# def create_line(point):
#     return LineString([fixed_point, point])

# # Apply the function to each geometry in the GeoDataFrame
# drawgeoframe['geom'] = drawgeoframe['center_geom'].apply(create_line)


# import branca.colormap as cm
# colormap = cm.LinearColormap(["green", "yellow", "red"], vmin=0, vmax=50)
# drawgeoframe["color"] = drawgeoframe["Count"].apply(lambda x: colormap(x)[:-2])

# start_lat = 48.8915079
# start_long = 2.3495425
# m = folium.Map(location=[start_lat, start_long], zoom_start=13)
# folium.GeoJson(drawgeoframe[["geometry", "color"]], style_function=lambda f: {"color": f['properties']['color']}).add_to(m)
# folium.GeoJson(drawgeoframe[["geom", "color"]].rename(columns={"geom": "geometry"}), style_function=lambda f: {"color": f['properties']['color']}).add_to(m)
# m

In [ ]:
# Group by h3 cell (level 4), count journeys ending in cell, and find common transportation mode for each cell
def most_common_value(series):
    return series.mode().iloc[0]
dfh3 = gdf_exit_idf.h3.geo_to_h3(4, lat_col="ending_latitude", lng_col="ending_longitude", set_index=False)

# Avoid counting the same user twice per cell
df_unique_user = dfh3.drop_duplicates(subset=['h3_04', 'user_id'])

# Do the grouping
drawgeoframe = df_unique_user[['h3_04', 'top_transportation_mode_tr']].groupby(['h3_04']).agg(Count=('h3_04', np.size), MostCommonTransport=('top_transportation_mode_tr', most_common_value))
drawgeoframe = drawgeoframe.reset_index().set_index('h3_04')

# Filter cells where we have less than 4 trips, to help with anonymity, and data quality
drawgeoframe = drawgeoframe[drawgeoframe['Count'] > 3]

# Find center of h3 cell and store the point in center_geom
drawgeoframe = drawgeoframe.h3.h3_to_geo()
drawgeoframe["center_geom"] = drawgeoframe["geometry"]

# Store the hexagon shape in "geometry" (default)
drawgeoframe = drawgeoframe.h3.h3_to_geo_boundary()

drawgeoframe

In [ ]:
fixed_point = Point(2.333333, 48.866667)

# Function to create a line from the fixed point to each point
def create_line(point):
    return LineString([fixed_point, point])

# Store in geom a line between the center of IDF and the center of the cell, this will be used for the lines viz
drawgeoframe['geom'] = drawgeoframe['center_geom'].apply(create_line)

In [ ]:
# Add color param, depending on the mode
colormap = {
    "PLANE": "red",
    "PASSENGER_CAR": "orange",
    "PASSENGER_CARFOOT": "darkorange",
    "RAIL_TRIP": "green",
    "HIGH_SPEED_TRAIN": "green",
    "HIGH_SPEED_TRAINRAIL_TRIP": "darkgreen",
    "PASSENGER_CARRAIL_TRIP": "yellow",
    "FOOT": "black" # wtf
}
drawgeoframe["color"] = drawgeoframe["MostCommonTransport"].apply(lambda x: colormap.get(x, "gray"))

# Save exode and exode_liness geojsons
drawgeoframe[["geometry", "color", "Count", "MostCommonTransport"]].to_file(EXODE_OUTPUT_FILE, driver="GeoJSON")
drawgeoframe[["geom", "color", "Count", "MostCommonTransport"]].rename(columns={"geom": "geometry"}).to_file(EXODE_LINES_OUTPUT_FILE, driver="GeoJSON")

# Locally display results, optionnal
start_lat = 48.8915079
start_long = 2.3495425
m = folium.Map(location=[start_lat, start_long], zoom_start=13)
folium.GeoJson(drawgeoframe[["geometry", "color"]], style_function=lambda f: {"color": f['properties']['color']}).add_to(m)
folium.GeoJson(drawgeoframe[["geom", "color"]].rename(columns={"geom": "geometry"}), style_function=lambda f: {"weight": "0.5", "color": f['properties']['color']}).add_to(m)
m

### Now we repeat the process for journeys coming to IDF (Called "Inxode")

In [ ]:
geofence_idf_geometry = geofence_idf.geometry

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.starting_longitude, df.starting_latitude), crs="EPSG:4326")

# Remove trips starting in idf
gdf_from_idf = gdf[~gdf.geometry.within(geofence_idf_geometry)]

# Create end_geometry for the end points
gdf_from_idf['end_geometry'] = gpd.points_from_xy(gdf_from_idf.ending_longitude, gdf_from_idf.ending_latitude)

# Keep only trips ending in idf
gdf_enter_idf = gdf_from_idf[gdf_from_idf['end_geometry'].within(geofence_idf_geometry)]

# Drop the temporary 'end_geometry' column if not needed
gdf_enter_idf = gdf_enter_idf.drop(columns='end_geometry')
gdf_enter_idf


In [ ]:
# Group by h3 cell (level 4), count journeys ending in cell, and find common transportation mode for each cell
def most_common_value(series):
    return series.mode().iloc[0]
dfh3 = gdf_enter_idf.h3.geo_to_h3(4, lat_col="starting_latitude", lng_col="starting_longitude", set_index=False)

# Avoid counting the same user twice per cell
df_unique_user = dfh3.drop_duplicates(subset=['h3_04', 'user_id'])

# Do the grouping
drawgeoframe = df_unique_user[['h3_04', 'top_transportation_mode_tr']].groupby(['h3_04']).agg(Count=('h3_04', np.size), MostCommonTransport=('top_transportation_mode_tr', most_common_value))
drawgeoframe = drawgeoframe.reset_index().set_index('h3_04')

# Filter cells where we have less than 4 trips, to help with anonymity, and data quality
drawgeoframe = drawgeoframe[drawgeoframe['Count'] > 3]

# Find center of h3 cell and store the point in center_geom
drawgeoframe = drawgeoframe.h3.h3_to_geo()
drawgeoframe["center_geom"] = drawgeoframe["geometry"]

# Store the hexagon shape in "geometry" (default)
drawgeoframe = drawgeoframe.h3.h3_to_geo_boundary()

drawgeoframe

In [ ]:
fixed_point = Point(2.333333, 48.866667)

# Function to create a line from the fixed point to each point
def create_line(point):
    return LineString([fixed_point, point])

# Store in geom a line between the center of IDF and the center of the cell, this will be used for the lines viz
drawgeoframe['geom'] = drawgeoframe['center_geom'].apply(create_line)

In [ ]:
# Add color param, depending on the mode
colormap = {
    "PLANE": "red",
    "PASSENGER_CAR": "orange",
    "RAIL_TRIP": "green",
    "HIGH_SPEED_TRAIN": "green",
    "FOOT": "black" # wtf
}
drawgeoframe["color"] = drawgeoframe["MostCommonTransport"].apply(lambda x: colormap.get(x))

# Save inxode and inxode_lines geojsons
drawgeoframe[["geometry", "color", "Count", "MostCommonTransport"]].to_file(INXODE_OUTPUT_FILE, driver="GeoJSON")
drawgeoframe[["geom", "color", "Count", "MostCommonTransport"]].rename(columns={"geom": "geometry"}).to_file(INXODE_LINES_OUTPUT_FILE, driver="GeoJSON")

# Locally display results, optionnal
start_lat = 48.8915079
start_long = 2.3495425
m = folium.Map(location=[start_lat, start_long], zoom_start=13)
folium.GeoJson(drawgeoframe[["geometry", "color"]], style_function=lambda f: {"color": f['properties']['color']}).add_to(m)
folium.GeoJson(drawgeoframe[["geom", "color", "Count"]].rename(columns={"geom": "geometry"}), style_function=lambda f: {"weight": int(f['properties']['Count'])/100, "color": f['properties']['color']}).add_to(m)
m